In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 330M/331M [00:15<00:00, 23.0MB/s]
100% 331M/331M [00:16<00:00, 21.6MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [5]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [6]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
len(age)

23708

In [8]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [9]:
df.shape

(23708, 3)

In [10]:
df.head()

,age,gender,img
0,10,1,10_1_0_20170109204259563.jpg.chip.jpg
1,35,1,35_1_0_20170117174927240.jpg.chip.jpg
2,47,0,47_0_3_20170117190238883.jpg.chip.jpg
3,49,1,49_1_0_20170104184001972.jpg.chip.jpg
4,40,0,40_0_0_20170117182249476.jpg.chip.jpg


In [11]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [12]:
train_df.shape

(20000, 3)

In [13]:
test_df.shape

(3708, 3)

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [16]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [17]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 5s 0us/step


In [18]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [19]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [20]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [21]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 270s 417ms/step - loss: 95.5520 - age_loss: 15.4884 - gender_loss: 0.8087 - age_mae: 15.4884 - gender_accuracy: 0.5134 - val_loss: 83.3480 - val_age_loss: 14.8291 - val_gender_loss: 0.6921 - val_age_mae: 14.8291 - val_gender_accuracy: 0.5229
Epoch 2/10
625/625 [==============================] - 251s 401ms/step - loss: 83.9261 - age_loss: 15.0072 - gender_loss: 0.6962 - age_mae: 15.0072 - gender_accuracy: 0.5214 - val_loss: 83.4716 - val_age_loss: 14.9105 - val_gender_loss: 0.6925 - val_age_mae: 14.9105 - val_gender_accuracy: 0.5229
Epoch 3/10
625/625 [==============================] - 248s 396ms/step - loss: 83.5462 - age_loss: 14.8675 - gender_loss: 0.6937 - age_mae: 14.8675 - gender_accuracy: 0.5227 - val_loss: 83.8375 - val_age_loss: 15.2926 - val_gender_loss: 0.6924 - val_age_mae: 15.2926 - val_gender_accuracy: 0.5229
Epoch 4/10
625/625 [==============================] - 258s 413ms/step - loss: 83.3590 - age_loss: 14.7562 - gend